# Init 

In [1]:
import pandas as pd

In [2]:
def fullDisplay(df,max_rows=None,max_col=None,width=None):
    df_cp = df.style.set_properties( **{'width': f'{width}px'}) if width is not None else df.copy() 
    with pd.option_context('display.max_rows', max_rows, 'display.max_columns', max_col,):
        display(df_cp)

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [4]:
def ndig_id(id):
    try:
        new_id = '_'.join([f'{int(i):04d}' for i in id.split('_')])
    except:
        new_id = 'ERROR'
    
    return new_id
    

# Create data-frame

In [11]:
fname = './AMISH_12192019.csv'#'./AMISH_reduced.csv'  #'./AMISH_OCT.csv'
amish_head = pd.read_csv(fname,nrows=2)

col_desc = {amish_head.iloc[1,i]:col.replace('\n',' ') for i, col in enumerate(amish_head.columns)}
col_val_desc = {amish_head.iloc[1,i]:str(vals).replace('\n',' ') for i, vals in enumerate(amish_head.iloc[0,:])}

amish_df = pd.read_csv(fname,skiprows=2,parse_dates=['EXAM_DATE'],dtype={'INDIVIDUAL':str},skipfooter=1)
amish_df['PAT_ID'] = amish_df.GROUP+'_'+amish_df.INDIVIDUAL
amish_df['PAT_ID'] = amish_df.PAT_ID.apply(ndig_id)
amish_df['CASE_ID'] = amish_df.PAT_ID +'_'+amish_df.EXAM_DATE.apply(lambda d: d.strftime("%m%d%Y"))

print(amish_df.shape)
amish_df.head()

(1249, 120)


/opt/data/anaconda3/envs/fastai4/lib/python3.7/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  import sys


,STUDY,SUBSTUDY,CENTER,GROUP,INDIVIDUAL,EXAM_DATE,EXAMINER,EYE,CO_COMMENTS_OD,CO_COMMENTS_OS,...,IRP_ACTION_OD,IRP_ACTION_OS,IRP_OPTH_FINDINGS_OD,IRP_OPTH_FINDINGS_OS,IRP_QIMG_OD,IRP_QIMG_OS,IRP_RP_OD,IRP_RP_OS,PAT_ID,CASE_ID
0,AMD,AMDAMISH,UPEN,120,5,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,NaN,NaN,A,A,N,N,0120_0005,0120_0005_08152013
1,AMD,AMDAMISH,UPEN,120,6,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,NaN,NaN,A,A,N,N,0120_0006,0120_0006_08152013
2,AMD,AMDAMISH,UPEN,120,7,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,NaN,NaN,A,A,N,N,0120_0007,0120_0007_08152013
3,AMD,AMDAMISH,UPEN,120,8,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,NaN,NaN,A,A,N,N,0120_0008,0120_0008_08152013
4,AMD,AMDAMISH,UPEN,158,10,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,NaN,NaN,A,A,N,N,0158_0010,0158_0010_08152013


In [ ]:
amish_df[]

In [13]:
print(amish_df.columns.to_list())

['STUDY', 'SUBSTUDY', 'CENTER', 'GROUP', 'INDIVIDUAL', 'EXAM_DATE', 'EXAMINER', 'EYE', 'CO_COMMENTS_OD', 'CO_COMMENTS_OS', 'CO_ACTION_OD', 'CO_ACTION_OS', 'CO_OPTH_FINDINGS_OD', 'CO_OPTH_FINDINGS_OS', 'CO_QIMG_OD', 'CO_QIMG_OS', 'CO_RPE_A3MM_OD', 'CO_RPE_A3MM_OS', 'CO_RPE_A5MM_OD', 'CO_RPE_A5MM_OS', 'CO_RPE_V3MM_OD', 'CO_RPE_V3MM_OS', 'CO_RPE_V5MM_OD', 'CO_RPE_V5MM_OS', 'CO_GA_A5MM_OD', 'CO_GA_A5MM_OS', 'CO_SUBFOVEAL_OD', 'CO_SUBFOVEAL_OS', 'CO_SUBRETINAL_OD', 'CO_SUBRETINAL_OS', 'CO_SRTSRHRM_OD', 'CO_SRTSRHRM_OS', 'CO_INTRA_RCS_OD', 'CO_INTRA_RCS_OS', 'CO_OUTER_RT_OD', 'CO_OUTER_RT_OS', 'CO_SR_DRUSEN_OD', 'CO_SR_DRUSEN_OS', 'CO_HRF_IRHRFOND_OD', 'CO_HRF_IRHRFOND_OS', 'CO_HRF_HRFOD_OD', 'CO_HRF_HRFOD_OS', 'CO_HRF_LHRFOD_OD', 'CO_HRF_LHRFOD_OS', 'CO_PED_DPED_OD', 'CO_PED_DPED_OS', 'CO_PED_HPED_OD', 'CO_PED_HPED_OS', 'CO_PED_SEROUS_OD', 'CO_PED_SEROUS_OS', 'SO_COMMENTS_OD', 'SO_COMMENTS_OS', 'SO_ACTION_OD', 'SO_ACTION_OS', 'SO_OPTH_FINDINGS_OD', 'SO_OPTH_FINDINGS_OS', 'SO_QIMG_OD', 'SO_Q

## add volume as a binary feature 

In [14]:
amish_df['CO_RPE_V3MM_OS'] = amish_df['CO_RPE_V3MM_OS'].replace('NE',-3).astype(np.float32)

In [15]:
def binarizeNumerical(p,th,df):
    df[f'{p}_L{th}_OD'] = (df[f'{p}_OD']>=th).values
    df[f'{p}_L{th}_OD'][df[f'{p}_OD']<0]=np.nan

    df[f'{p}_L{th}_OS'] = (df[f'{p}_OS']>=th).values
    df[f'{p}_L{th}_OS'][df[f'{p}_OS']<0]=np.nan
    


In [16]:
th = 0.03 # <= 0.03mm^2
p = 'CO_RPE_V3MM' # Drusen Volume

binarizeNumerical(p,0.03,amish_df)

/opt/data/anaconda3/envs/fastai4/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/data/anaconda3/envs/fastai4/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
hypo_df = pd.read_csv('AMISH_hypoDrusen.csv',nrows=None,dtype={'INDIVIDUAL':object},parse_dates=['EXAM_DATE'])
hypo_df.drop(['STUDY','SUBSTUDY','EXAMINER'],axis=1,inplace=True)
hypo_df

,CENTER,GROUP,INDIVIDUAL,EXAM_DATE,CO_Drusen_Core_OD,CO_Drusen_Core_OS,Notes
0,UPEN,120,5,2013-08-15,NE,NE,NaN
1,UPEN,193,4,2013-08-15,NE,NE,NaN
2,UPEN,193,10,2013-08-15,NE,NE,NaN
3,UPEN,498/25-6,2,2013-08-22,Y,Y,NaN
4,IHG,5001,100,2013-10-29,N,N,NaN
...,...,...,...,...,...,...,...
224,UPEN,585,6,2016-04-07,N,N,NaN
225,UPEN,585,7,2016-04-07,N,N,NaN
226,UPEN,291,9,2016-04-22,N,N,NaN
227,UPEN,219,3,2016-06-02,N,N,NaN


In [18]:
amish_df

,STUDY,SUBSTUDY,CENTER,GROUP,INDIVIDUAL,EXAM_DATE,EXAMINER,EYE,CO_COMMENTS_OD,CO_COMMENTS_OS,...,IRP_OPTH_FINDINGS_OD,IRP_OPTH_FINDINGS_OS,IRP_QIMG_OD,IRP_QIMG_OS,IRP_RP_OD,IRP_RP_OS,PAT_ID,CASE_ID,CO_RPE_V3MM_L0.03_OD,CO_RPE_V3MM_L0.03_OS
0,AMD,AMDAMISH,UPEN,120,5,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,0120_0005,0120_0005_08152013,0.0,0.0
1,AMD,AMDAMISH,UPEN,120,6,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,0120_0006,0120_0006_08152013,0.0,0.0
2,AMD,AMDAMISH,UPEN,120,7,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,0120_0007,0120_0007_08152013,0.0,0.0
3,AMD,AMDAMISH,UPEN,120,8,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,0120_0008,0120_0008_08152013,0.0,0.0
4,AMD,AMDAMISH,UPEN,158,10,2013-08-15,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,0158_0010,0158_0010_08152013,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244,AMD,AMDAMISH,VAN,4000,14206,2016-12-23,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,4000_14206,4000_14206_12232016,0.0,0.0
1245,AMD,AMDAMISH,VAN,4000,14129,2017-01-26,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,4000_14129,4000_14129_01262017,0.0,0.0
1246,AMD,AMDAMISH,VAN,4000,14126,2017-01-26,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,N,N,4000_14126,4000_14126_01262017,0.0,0.0
1247,AMD,AMDAMISH,VAN,4000,3976,2017-01-30,Muneeswar,B,NaN,NaN,...,NaN,NaN,A,A,Y,Y,4000_3976,4000_3976_01302017,0.0,0.0


In [19]:
amish_df_tmp = pd.merge(amish_df,hypo_df,on=['CENTER','GROUP','INDIVIDUAL','EXAM_DATE'],how='left')
amish_df_tmp[['CO_Drusen_Core_OD','CO_Drusen_Core_OS']] = amish_df_tmp[['CO_Drusen_Core_OD','CO_Drusen_Core_OS']].fillna('N')

In [20]:
amish_df = amish_df_tmp

## external df 

In [23]:
amish_df.head(5)

,STUDY,SUBSTUDY,CENTER,GROUP,INDIVIDUAL,EXAM_DATE,EXAMINER,EYE,CO_QIMG_OD,CO_QIMG_OS,...,SO_HRF_LHRFOD_OD,SO_HRF_LHRFOD_OS,CSLO_RP_OD,CSLO_RP_OS,IRP_QIMG_OD,IRP_QIMG_OS,IRP_RP_OD,IRP_RP_OS,PAT_ID,CASE_ID
0,AMD,AMDAMISH,UPEN,120,5,2013-08-15,Muneeswar,B,A,A,...,NaN,NaN,N,N,A,A,N,N,0120_0005,0120_0005_08152013
1,AMD,AMDAMISH,UPEN,120,6,2013-08-15,Muneeswar,B,A,A,...,NaN,NaN,N,N,A,A,N,N,0120_0006,0120_0006_08152013
2,AMD,AMDAMISH,UPEN,120,7,2013-08-15,Muneeswar,B,A,A,...,NaN,NaN,N,N,A,A,N,N,0120_0007,0120_0007_08152013
3,AMD,AMDAMISH,UPEN,120,8,2013-08-15,Muneeswar,B,A,A,...,NaN,NaN,N,N,A,A,N,N,0120_0008,0120_0008_08152013
4,AMD,AMDAMISH,UPEN,158,10,2013-08-15,Muneeswar,B,A,A,...,NaN,NaN,N,N,A,A,N,N,0158_0010,0158_0010_08152013


In [28]:
from pathlib import Path
ext_path = Path('/opt/data/Jupyter_Notebook/NadavRakocz/Doheny/reproduce/')
ext_files = [d for d in ext_path.iterdir() if 'csv' in d.name]
ext_files

In [36]:
f.stem

'AMISH_01-24-2020_Swetha'

In [38]:
ext_dfs = []
for f in ext_files:
    o_name = f'{f.stem}_ext.csv'
    ext_amish_df = pd.read_csv(f,skiprows=2,parse_dates=['EXAM_DATE'],dtype={'INDIVIDUAL':str},skipfooter=1)
    ext_amish_df['PAT_ID'] = ext_amish_df.ID
    ext_amish_df['PAT_ID'] = ext_amish_df.PAT_ID.apply(ndig_id)
    ext_amish_df['CASE_ID'] = ext_amish_df.PAT_ID +'_'+ext_amish_df.EXAM_DATE.apply(lambda d: d.strftime("%m%d%Y"))
    # binarizeNumerical('CO_RPE_V3MM',0.03,ext_amish_df)
    ext_dfs.append(ext_amish_df)
    

/opt/data/anaconda3/envs/fastai4/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [25]:
a=ext_amish_df[ext_amish_df['CENTER']=='UPEN']
a.sort_values(by='EXAM_DATE')

,CENTER,ID,EXAM_DATE,EXAMINER,EYE,SO_COMMENTS_OD,SO_COMMENTS_OS,SO_ACTION_OD,SO_ACTION_OS,SO_OPTH_FINDINGS_OD,...,SO_HRF_LHRFOD_OD,SO_HRF_LHRFOD_OS,SO_PED_DPED_OD,SO_PED_DPED_OS,SO_PED_HPED_OD,SO_PED_HPED_OS,SO_PED_SEROUS_OD,SO_PED_SEROUS_OS,PAT_ID,CASE_ID
25,UPEN,350_10,2015-08-05,LT,NaN,NaN,NaN,NaN,NaN,DRUSEN,...,NE,NE,N,N,N,N,N,N,0350_0010,0350_0010_08052015
26,UPEN,327_8,2015-10-22,LT,NaN,NaN,NaN,NaN,NaN,NaN,...,NE,NE,N,N,N,N,N,N,0327_0008,0327_0008_10222015
27,UPEN,342_11,2015-10-22,LT,NaN,NaN,NaN,NaN,NaN,MYOPIA,...,NE,NE,N,N,N,N,N,N,0342_0011,0342_0011_10222015
28,UPEN,327_5,2015-10-22,LT,NaN,NaN,NaN,NaN,NaN,NaN,...,NE,NE,N,N,N,N,N,N,0327_0005,0327_0005_10222015
29,UPEN,311_3,2015-10-29,LT,NaN,NaN,NaN,NaN,NaN,ERM,...,NE,NE,N,N,N,N,N,N,0311_0003,0311_0003_10292015
30,UPEN,334_9,2015-10-29,LT,NaN,NaN,NaN,NaN,NaN,NaN,...,NE,NE,NE,N,NE,N,NE,N,0334_0009,0334_0009_10292015
31,UPEN,311_3,2015-10-29,LT,NaN,NaN,NaN,NaN,NaN,ERM,...,NE,NE,N,N,N,N,N,N,0311_0003,0311_0003_10292015
32,UPEN,327_6,2015-10-29,LT,NaN,NaN,NaN,NaN,NaN,NaN,...,ONL,OPL,Y,Y,N,N,N,N,0327_0006,0327_0006_10292015
33,UPEN,139_11,2015-12-10,LT,NaN,NaN,NaN,NaN,NaN,NaN,...,NE,NE,N,N,N,N,N,N,0139_0011,0139_0011_12102015
34,UPEN,46_7,2015-12-10,LT,NaN,NaN,NaN,NaN,NaN,MYOPIA,...,NE,NE,Y,N,N,N,N,N,0046_0007,0046_0007_12102015


## Exclude bad images 

In [39]:
exclude = ['P','CG','NA','NE']
exclude_ish = ['CG','NA','NE']

In [22]:
amish_df_qc = amish_df[~(amish_df.SO_QIMG_OD.isin(exclude) |  amish_df.SO_QIMG_OS.isin(exclude))]
amish_df_q = amish_df[~(amish_df.SO_QIMG_OD.isin(exclude_ish) |  amish_df.SO_QIMG_OS.isin(exclude_ish))]

In [40]:
ext_amish_df_qc = ext_amish_df[~(ext_amish_df.SO_QIMG_OD.isin(exclude_ish) |  ext_amish_df.SO_QIMG_OS.isin(exclude_ish))]
ext_amish_df_qc

,CENTER,GROUP,INDIVIDUAL,ID,EXAM_DATE,EXAMINER,SO_COMMENTS_OD,SO_COMMENTS_OS,SO_OPTH_FINDINGS_OD,SO_OPTH_FINDINGS_OS,...,SO_HRF_LHRFOD_OD,SO_HRF_LHRFOD_OS,SO_PED_DPED_OD,SO_PED_DPED_OS,SO_PED_HPED_OD,SO_PED_HPED_OS,SO_PED_SEROUS_OD,SO_PED_SEROUS_OS,PAT_ID,CASE_ID
0,UPEN,228,4,228_4,2015-08-13,LT,NaN,NaN,NaN,NaN,...,N,IS/OS,Y,Y,N,N,N,N,0228_0004,0228_0004_08132015
1,UPEN,307,6,307_6,2015-08-13,LT,NaN,NaN,NaN,NaN,...,ONL,OPL,Y,N,N,N,N,N,0307_0006,0307_0006_08132015
2,UPEN,269,8,269_8,2015-10-01,LT,NaN,NaN,NaN,NaN,...,OPL,OPL,Y,Y,N,N,Y,Y,0269_0008,0269_0008_10012015
3,UPEN,139,4,139_4,2015-12-10,LT,NaN,NaN,NaN,NaN,...,IS/OS,OPL,N,Y,N,N,N,N,0139_0004,0139_0004_12102015
4,UPEN,202,3,202_3,2015-12-10,LT,NaN,NaN,NaN,ERM,...,NE,NE,N,N,N,N,N,N,0202_0003,0202_0003_12102015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,UPEN,286,5,286_5,2016-03-14,LT,NaN,NaN,NaN,NaN,...,NE,NE,N,N,N,N,N,N,0286_0005,0286_0005_03142016
104,UPEN,302,8,302_8,2016-03-14,LT,NaN,NaN,ERM,NaN,...,NE,NE,N,N,N,N,N,N,0302_0008,0302_0008_03142016
105,UPEN,539,5,539_5,2016-03-14,LT,NaN,NaN,NaN,NaN,...,NE,ONL,N,Y,N,N,N,N,0539_0005,0539_0005_03142016
106,UPEN,105,11,105_11,2016-03-14,LT,NaN,NaN,NaN,NaN,...,NE,NE,N,N,N,N,N,N,0105_0011,0105_0011_03142016


## Save to disc 

In [23]:
amish_df.columns.to_list()

['STUDY',
 'SUBSTUDY',
 'CENTER',
 'GROUP',
 'INDIVIDUAL',
 'EXAM_DATE',
 'EXAMINER',
 'EYE',
 'CO_COMMENTS_OD',
 'CO_COMMENTS_OS',
 'CO_ACTION_OD',
 'CO_ACTION_OS',
 'CO_OPTH_FINDINGS_OD',
 'CO_OPTH_FINDINGS_OS',
 'CO_QIMG_OD',
 'CO_QIMG_OS',
 'CO_RPE_A3MM_OD',
 'CO_RPE_A3MM_OS',
 'CO_RPE_A5MM_OD',
 'CO_RPE_A5MM_OS',
 'CO_RPE_V3MM_OD',
 'CO_RPE_V3MM_OS',
 'CO_RPE_V5MM_OD',
 'CO_RPE_V5MM_OS',
 'CO_GA_A5MM_OD',
 'CO_GA_A5MM_OS',
 'CO_SUBFOVEAL_OD',
 'CO_SUBFOVEAL_OS',
 'CO_SUBRETINAL_OD',
 'CO_SUBRETINAL_OS',
 'CO_SRTSRHRM_OD',
 'CO_SRTSRHRM_OS',
 'CO_INTRA_RCS_OD',
 'CO_INTRA_RCS_OS',
 'CO_OUTER_RT_OD',
 'CO_OUTER_RT_OS',
 'CO_SR_DRUSEN_OD',
 'CO_SR_DRUSEN_OS',
 'CO_HRF_IRHRFOND_OD',
 'CO_HRF_IRHRFOND_OS',
 'CO_HRF_HRFOD_OD',
 'CO_HRF_HRFOD_OS',
 'CO_HRF_LHRFOD_OD',
 'CO_HRF_LHRFOD_OS',
 'CO_PED_DPED_OD',
 'CO_PED_DPED_OS',
 'CO_PED_HPED_OD',
 'CO_PED_HPED_OS',
 'CO_PED_SEROUS_OD',
 'CO_PED_SEROUS_OS',
 'SO_COMMENTS_OD',
 'SO_COMMENTS_OS',
 'SO_ACTION_OD',
 'SO_ACTION_OS',
 'SO_OPTH_FI

In [25]:
AMISH_FNAME = 'AMISH_12192019_wHypoDrusen'
amish_df_qc.to_csv(f'{AMISH_FNAME}.csv',index=False)
amish_df_q.to_csv(f'{AMISH_FNAME}_wQ.csv',index=False)


### ext_df 

In [29]:
ext_amish_df = ext_amish_df.to_csv('EXT_AMISH_SO_QC.csv',index=False)

In [41]:
for f,ext_df in zip(ext_files,ext_dfs):
    o_name = f'{f.stem}_ext_qc.csv'
    ext_df.to_csv(o_name)

# Review features 

In [8]:
col,desc = zip(*[(k,col_desc[k]) for k in col_desc.keys()])
col,vals = zip(*[(k,col_val_desc[k]) for k in col_val_desc.keys()])


a = pd.DataFrame({'COL':col,'DESC':desc})
b = pd.DataFrame({'COL':col,'VALS':vals})
cols_df = pd.merge(a,b,on='COL',how='outer')

fullDisplay(cols_df,width=600)

,COL,DESC,VALS
0,STUDY,AMD Study Data CRF Questions,Choice Answers
1,SUBSTUDY,Unnamed: 1,nan
2,CENTER,Unnamed: 2,nan
3,GROUP,Unnamed: 3,nan
4,INDIVIDUAL,Unnamed: 4,nan
5,EXAM_DATE,Unnamed: 5,nan
6,EXAMINER,Unnamed: 6,nan
7,EYE,Is this form being completed for,Both (B) Right (R) Left (L)
8,CO_COMMENTS_OD,CIRRUS OCT Comments on data OD,Text (50)
9,CO_COMMENTS_OS,CIRRUS OCT Comments on data OS,Text (50)


In [12]:
amish_df[['CO_RPE_V5MM_OD','CO_RPE_V3MM_OS']].max()

CO_RPE_V5MM_OD    2.19
CO_RPE_V3MM_OS    1.04
dtype: float64

# SO

In [9]:
so_col = [c for c in amish_df.columns.to_list() if 'SO' in c]
fullDisplay(amish_df[so_col],max_rows=10)

,SO_COMMENTS_OD,SO_COMMENTS_OS,SO_ACTION_OD,SO_ACTION_OS,SO_OPTH_FINDINGS_OD,SO_OPTH_FINDINGS_OS,SO_QIMG_OD,SO_QIMG_OS,SO_RPE_A3MM_OD,SO_RPE_A3MM_OS,SO_RPE_A5MM_OD,SO_RPE_A5MM_OS,SO_RPE_V3MM_OD,SO_RPE_V3MM_OS,SO_RPE_V5MM_OD,SO_RPE_V5MM_OS,SO_GA_A5MM_OD,SO_GA_A5MM_OS,SO_SUBFOVEAL_OD,SO_SUBFOVEAL_OS,SO_SUBRETINAL_OD,SO_SUBRETINAL_OS,SO_SRTSRHRM_OD,SO_SRTSRHRM_OS,SO_INTRA_RCS_OD,SO_INTRA_RCS_OS,SO_OUTER_RT_OD,SO_OUTER_RT_OS,SO_SR_DRUSEN_OD,SO_SR_DRUSEN_OS,SO_HRF_IRHRFOND_OD,SO_HRF_IRHRFOND_OS,SO_HRF_HRFOD_OD,SO_HRF_HRFOD_OS,SO_HRF_LHRFOD_OD,SO_HRF_LHRFOD_OS,SO_PED_DPED_OD,SO_PED_DPED_OS,SO_PED_HPED_OD,SO_PED_HPED_OS,SO_PED_SEROUS_OD,SO_PED_SEROUS_OS
0,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,-1,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,Y,Y,N,N,N,N
1,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,315,-1,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
2,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,393,297,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
3,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,305,315,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
4,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,302,308,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,200,183,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
1327,NaN,NaN,NaN,NaN,vitelliform lision,vitelliform lision,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,-1,279,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
1328,NaN,NaN,NaN,NaN,"ERM, PSEUDO HOLE, CYSTS",NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,158,253,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N
1329,NaN,NaN,NaN,NaN,NaN,NaN,A,A,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,248,248,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,N,N,N,N,N,N


In [77]:
print(cols_df.COL.iloc[68:84])
print(cols_df.COL.iloc[86:])



68       SO_SUBFOVEAL_OD
69       SO_SUBFOVEAL_OS
70      SO_SUBRETINAL_OD
71      SO_SUBRETINAL_OS
72        SO_SRTSRHRM_OD
73        SO_SRTSRHRM_OS
74       SO_INTRA_RCS_OD
75       SO_INTRA_RCS_OS
76        SO_OUTER_RT_OD
77        SO_OUTER_RT_OS
78       SO_SR_DRUSEN_OD
79       SO_SR_DRUSEN_OS
80    SO_HRF_IRHRFOND_OD
81    SO_HRF_IRHRFOND_OS
82       SO_HRF_HRFOD_OD
83       SO_HRF_HRFOD_OS
Name: COL, dtype: object
86      SO_PED_DPED_OD
87      SO_PED_DPED_OS
88      SO_PED_HPED_OD
89      SO_PED_HPED_OS
90    SO_PED_SEROUS_OD
91    SO_PED_SEROUS_OS
Name: COL, dtype: object


In [83]:
[c for c,v in col_val_desc.items() if 'No (N)' in v]

['CO_SUBRETINAL_OD',
 'CO_SUBRETINAL_OS',
 'CO_SRTSRHRM_OD',
 'CO_SRTSRHRM_OS',
 'CO_INTRA_RCS_OD',
 'CO_INTRA_RCS_OS',
 'CO_OUTER_RT_OD',
 'CO_OUTER_RT_OS',
 'CO_SR_DRUSEN_OD',
 'CO_SR_DRUSEN_OS',
 'CO_HRF_IRHRFOND_OD',
 'CO_HRF_IRHRFOND_OS',
 'CO_HRF_HRFOD_OD',
 'CO_HRF_HRFOD_OS',
 'CO_PED_DPED_OD',
 'CO_PED_DPED_OS',
 'CO_PED_HPED_OD',
 'CO_PED_HPED_OS',
 'CO_PED_SEROUS_OD',
 'CO_PED_SEROUS_OS',
 'SO_SUBRETINAL_OD',
 'SO_SUBRETINAL_OS',
 'SO_SRTSRHRM_OD',
 'SO_SRTSRHRM_OS',
 'SO_INTRA_RCS_OD',
 'SO_INTRA_RCS_OS',
 'SO_OUTER_RT_OD',
 'SO_OUTER_RT_OS',
 'SO_SR_DRUSEN_OD',
 'SO_SR_DRUSEN_OS',
 'SO_HRF_IRHRFOND_OD',
 'SO_HRF_IRHRFOND_OS',
 'SO_HRF_HRFOD_OD',
 'SO_HRF_HRFOD_OS',
 'SO_PED_DPED_OD',
 'SO_PED_DPED_OS',
 'SO_PED_HPED_OD',
 'SO_PED_HPED_OS',
 'SO_PED_SEROUS_OD',
 'SO_PED_SEROUS_OS']

In [10]:
['SO_SUBRETINAL_OD',
 'SO_SUBRETINAL_OS',
 'SO_SRTSRHRM_OD',
 'SO_SRTSRHRM_OS',
 'SO_INTRA_RCS_OD',
 'SO_INTRA_RCS_OS',
 'SO_OUTER_RT_OD',
 'SO_OUTER_RT_OS',
 'SO_SR_DRUSEN_OD',
 'SO_SR_DRUSEN_OS',
 'SO_HRF_IRHRFOND_OD',
 'SO_HRF_IRHRFOND_OS',
 'SO_HRF_HRFOD_OD',
 'SO_HRF_HRFOD_OS',
 'SO_PED_DPED_OD',
 'SO_PED_DPED_OS',
 'SO_PED_HPED_OD',
 'SO_PED_HPED_OS',
 'SO_PED_SEROUS_OD',
 'SO_PED_SEROUS_OS']


['SO_SUBRETINAL_OD',
 'SO_SUBRETINAL_OS',
 'SO_SRTSRHRM_OD',
 'SO_SRTSRHRM_OS',
 'SO_INTRA_RCS_OD',
 'SO_INTRA_RCS_OS',
 'SO_OUTER_RT_OD',
 'SO_OUTER_RT_OS',
 'SO_SR_DRUSEN_OD',
 'SO_SR_DRUSEN_OS',
 'SO_HRF_IRHRFOND_OD',
 'SO_HRF_IRHRFOND_OS',
 'SO_HRF_HRFOD_OD',
 'SO_HRF_HRFOD_OS',
 'SO_PED_DPED_OD',
 'SO_PED_DPED_OS',
 'SO_PED_HPED_OD',
 'SO_PED_HPED_OS',
 'SO_PED_SEROUS_OD',
 'SO_PED_SEROUS_OS']

In [12]:
len(so_col)//2

21